In [74]:
import nilearn
from nilearn import datasets, plotting
import os
import requests
import csv
import pandas as pd
import nibabel as nib
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
import scipy
from scipy.stats import multivariate_normal
from scipy.spatial.distance import pdist, squareform
from networkx.drawing.nx_agraph import graphviz_layout
from sklearn.metrics.cluster import mutual_info_score
from scipy.sparse.csgraph import minimum_spanning_tree
from collections import deque, defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

In [75]:
home_base_dir = '/Users/aj/dmello_lab/fmri_connectivity_trees' # directory where repository lives at home computer
lab_base_dir = '/Users/ajjain/Downloads/Code/fmri_connectivity_trees' # directory where repository lives at lab computer

# set base directory depending on where the code is being run
base_dir = home_base_dir if os.path.exists(home_base_dir) else lab_base_dir

# get msdl and whole brain atlases and coords
msdl_data = datasets.fetch_atlas_msdl()
msdl_coords = msdl_data.region_coords

# path for shapes and pooled timeseries
cort_shape_path = f'{base_dir}/code/functional_connectivity/abide/output/roi_time_series/884_MSDL/shape'
cort_pooled_path = f'{base_dir}/code/functional_connectivity/abide/output/roi_time_series/884_MSDL/pooled'

# load abide ids
with open(f'{base_dir}/datasets/abide/phenotypic/abide_ids.txt', 'r') as f:
    abide_ids = [line.strip() for line in f.readlines()]

# load mapped phenotypes
with open(f'{base_dir}/datasets/abide/phenotypic/phenotype.txt', 'r') as f:
    phenotype = [int(line.strip()) for line in f.readlines()]

# load saved covariance for entire dataset
asd_cov = pd.read_csv(f'{base_dir}/code/functional_connectivity/abide/output/covariance/msdl/asd_cov_msdl.csv').to_numpy()
tdc_cov = pd.read_csv(f'{base_dir}/code/functional_connectivity/abide/output/covariance/msdl/tdc_cov_msdl.csv').to_numpy()

# load saved mutual information for entire dataset
asd_mi = pd.read_csv(f'{base_dir}/code/functional_connectivity/abide/output/mutual_info/msdl/1000_bins/asd_mutual_info.csv').to_numpy()
tdc_mi = pd.read_csv(f'{base_dir}/code/functional_connectivity/abide/output/mutual_info/msdl/1000_bins/tdc_mutual_info.csv').to_numpy()

[get_dataset_dir] Dataset found in /Users/aj/nilearn_data/msdl_atlas


In [76]:
# separate timeseries for asd and tdc
def get_groups(abide_ids, phenotype):
    asd = []
    tdc = []
    for i in range(len(abide_ids)):
        if phenotype[i] == 1:
            asd.append(abide_ids[i])
        else: tdc.append(abide_ids[i])
    asd = np.array(asd)
    tdc = np.array(tdc)

    return {'asd': asd, 'tdc': tdc}

# concatenate timeseries to get a single timeseries for each subject
def get_timeseries(ids, cort_shape_path=cort_shape_path, cort_pooled_path=cort_pooled_path):
    timeseries = []
    for id in ids:

        # load the shape and pooled timeseries
        shape = np.loadtxt(f'{cort_shape_path}/{id}.csv', delimiter=',').astype(int)
        pooled = np.loadtxt(f'{cort_pooled_path}/{id}.csv', delimiter=',').reshape(shape)

        # concatenate the timeseries
        timeseries.append(pooled)

    return timeseries

# concatenate into one time series
def concat(timeseries):
    concat_timeseries = timeseries[0]
    for series in timeseries[1:]:
        concat_timeseries = np.concatenate((concat_timeseries, series), axis=0)
    return concat_timeseries

# get separate ids for each group
abide_groups = get_groups(abide_ids, phenotype)

# get timeseries for each group
asd_timeseries = get_timeseries(abide_groups['asd']) # asd timeseries
tdc_timeseries = get_timeseries(abide_groups['tdc']) # tdc timeseries

# all timeseries concatenated
asd_concat = concat(asd_timeseries) # asd concatenated timeseries
tdc_concat = concat(tdc_timeseries) # tdc concatenated timeseries

In [77]:
# get covariance matrix for a given timeseries
def get_covariance(concat_timeseries):
    """
    Get the covariance matrix of the concatenated timeseries.
    """
    # get the covariance matrix
    cov = np.cov(concat_timeseries.T)

    for i in range(len(cov)):
        for j in range(len(cov)):
            if i == j:
                cov[i][j] = 0

    return cov

# helper function for mutual info between two continuous random variables
def mutual_information_continuous(x, y, bins=100):
    """
    Estimate mutual information between two continuous variables by discretizing them.

    Parameters:
    x (array-like): First continuous variable.
    y (array-like): Second continuous variable.
    bins (int): Number of bins to use for discretization.

    Returns:
    float: Estimated mutual information.
    """
    # Convert to pandas Series
    x = pd.Series(x)
    y = pd.Series(y)

    # Discretize the continuous values
    x_binned = pd.cut(x, bins=bins, labels=False)
    y_binned = pd.cut(y, bins=bins, labels=False)

    # Compute contingency table
    contingency_table = pd.crosstab(x_binned, y_binned)

    # Compute mutual information
    mi = mutual_info_score(None, None, contingency=contingency_table.values)

    return mi

# get mutual information between all regions for a given timeseries
def get_mutual_info(timeseries, bins=1000):

    mutual_info_matrix = np.zeros((timeseries.shape[1], timeseries.shape[1]))

    # get mutual information
    for region in range(timeseries.shape[1]):
        for region2 in range(timeseries.shape[1]):
            if region != region2:
                mutual_info = mutual_information_continuous(timeseries[:, region], timeseries[:, region2], bins=bins)
                mutual_info_matrix[region, region2] = mutual_info
    
    return mutual_info_matrix

# Construct the Chow-Liu tree from continuous data using mutual information.
def get_chow_liu_tree(timeseries, mi_matrix=None, bins=1000):
    
    # Compute the mutual information matrix
    if mi_matrix is None:
        mi_matrix = get_mutual_info(timeseries, bins=bins)

    # Set the diagonal to zero (self-information)
    np.fill_diagonal(mi_matrix, 0)

    # We use the negative MI because scipy's minimum_spanning_tree computes *minimum* tree.
    mst = minimum_spanning_tree(-mi_matrix).toarray()

    # Make MST undirected and remove negative sign
    mst = -mst + (-mst).T

    # Create NetworkX graph for visualization or further use
    G = nx.Graph()
    n = mi_matrix.shape[1]
    for i in range(n):
        for j in range(i+1, n):
            if mst[i, j] != 0:
                G.add_edge(i, j, weight=mi_matrix[i, j])
    
    return G, mst

In [78]:
# truncate timeseries for prediction dataset
def truncate_timeseries(tdc_timeseries, asd_timeseries, fraction=0.1):

    n_tdc = int(len(tdc_timeseries) * fraction)
    n_asd = int(len(asd_timeseries) * fraction)

    # select n_tdc random timeseries
    tdc_indices = np.random.choice(len(tdc_timeseries), n_tdc, replace=False)
    tdc_timeseries_trunc = [tdc_timeseries[i] for i in tdc_indices]

    # select n_asd random timeseries
    asd_indices = np.random.choice(len(asd_timeseries), n_asd, replace=False)
    asd_timeseries_trunc = [asd_timeseries[i] for i in asd_indices]

    return tdc_timeseries_trunc, asd_timeseries_trunc

# get cross validation split
def cross_validation_split(tdc_timeseries_trunc, asd_timeseries_trunc, hold_out = 0.2):

    # initialize training and test sets
    tdc_training_set = []
    tdc_test_set = []
    asd_training_set = []
    asd_test_set = []

    # get number of training and test samples
    tdc_train_num = int(len(tdc_timeseries_trunc) * (1 - hold_out))
    asd_train_num = int(len(asd_timeseries_trunc) * (1 - hold_out))

    # randomly shuffle timeseries
    tdc_indices = np.random.choice(len(tdc_timeseries_trunc), len(tdc_timeseries_trunc), replace=False)
    asd_indices = np.random.choice(len(asd_timeseries_trunc), len(asd_timeseries_trunc), replace=False)

    # split into training and test sets
    tdc_train_indices = tdc_indices[:tdc_train_num]
    tdc_test_indices = tdc_indices[tdc_train_num:]
    asd_train_indices = asd_indices[:asd_train_num]
    asd_test_indices = asd_indices[asd_train_num:]

    # get training and test sets
    for i in tdc_train_indices:
        tdc_training_set.append(tdc_timeseries_trunc[i])
    for i in tdc_test_indices:
        tdc_test_set.append(tdc_timeseries_trunc[i])
    for i in asd_train_indices:
        asd_training_set.append(asd_timeseries_trunc[i])
    for i in asd_test_indices:
        asd_test_set.append(asd_timeseries_trunc[i])
    
    # return concatenated timeseries for each
    tdc_train_concat = concat(tdc_training_set)
    tdc_test_concat = concat(tdc_test_set)
    asd_train_concat = concat(asd_training_set)
    asd_test_concat = concat(asd_test_set)

    data = {"tdc_train": tdc_train_concat, "tdc_test": tdc_test_concat, "asd_train": asd_train_concat, "asd_test": asd_test_concat}
    return data

# separate concat timeseries into bins to create train and test set
def bin_data(data, bin_num = 100, hold_out = 0.2):

    train_bin_num = int(bin_num * (1 - hold_out))
    test_bin_num = int(bin_num * hold_out)

    # bin data
    tdc_train_bins = np.array_split(data['tdc_train'], train_bin_num)
    tdc_test_bins = np.array_split(data['tdc_test'], test_bin_num)
    asd_train_bins = np.array_split(data['asd_train'], train_bin_num)
    asd_test_bins = np.array_split(data['asd_test'], test_bin_num)

    # return data
    data = {"tdc_train": tdc_train_bins, "tdc_test": tdc_test_bins, "asd_train": asd_train_bins, "asd_test": asd_test_bins}
    return data

# consolidate
def data_split(tdc_timeseries, asd_timeseries, fraction=0.1, hold_out=0.2, bin_num=100):

    # truncate timeseries
    tdc_timeseries_trunc, asd_timeseries_trunc = truncate_timeseries(tdc_timeseries, asd_timeseries, fraction=fraction)

    # get cross validation split
    data = cross_validation_split(tdc_timeseries_trunc, asd_timeseries_trunc, hold_out=hold_out)

    # bin data
    data = bin_data(data, bin_num=bin_num, hold_out=hold_out)

    return data

# shuffle asd and tdc, assign labels
def shuffle_labels(data):

    # shuffle the labels
    tdc_train_labels = np.zeros(len(data['tdc_train']))
    tdc_test_labels = np.zeros(len(data['tdc_test']))
    asd_train_labels = np.ones(len(data['asd_train']))
    asd_test_labels = np.ones(len(data['asd_test']))

    # concatenate the data and labels
    train_data = data['tdc_train'] + data['asd_train']
    test_data = data['tdc_test'] + data['asd_test']
    train_labels = np.concatenate((tdc_train_labels, asd_train_labels), axis=0)
    test_labels = np.concatenate((tdc_test_labels, asd_test_labels), axis=0)

    # shuffle the data and labels
    indices = np.arange(len(train_data))
    np.random.shuffle(indices)
    train_data = [train_data[i] for i in indices]
    train_labels = train_labels[indices]

    indices = np.arange(len(test_data))
    np.random.shuffle(indices)
    test_data = [test_data[i] for i in indices]
    test_labels = test_labels[indices]
    
    data = {"train_data": train_data, "test_data": test_data, "train_labels": train_labels, "test_labels": test_labels}
    return data

In [95]:
def binary_prediction_model(data, model_name="gnb", model=GaussianNB(), metric="covariance", fraction=0.1, mi_bin_num=100):

    # get the data
    train_data = data['train_data']
    test_data = data['test_data']
    train_labels = data['train_labels']
    test_labels = data['test_labels']

    # calculate metric (feature extraction)
    if metric == "covariance":
        if os.path.exists(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/covariance/{fraction}/data.csv'):
            print("Using precomputed covariance matrices.")
            # load precomputed covariance matrices
            train_data = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/covariance/{fraction}/data.csv', delimiter=',')
            test_data = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/covariance/{fraction}/test_data.csv', delimiter=',')
            train_labels = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/covariance/{fraction}/train_labels.csv', delimiter=',')
            test_labels = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/covariance/{fraction}/test_labels.csv', delimiter=',')
        # if not, calculate covariance matrices
        else:
            print("Calculating covariance matrices.")
            train_data = np.array([get_covariance(series).flatten() for series in train_data])
            test_data = np.array([get_covariance(series).flatten() for series in test_data])
    
    elif metric == "mutual_info":
        if os.path.exists(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/data.csv'):
            print("Using precomputed mutual information.")
            # load precomputed mutual information
            train_data = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/data.csv', delimiter=',')
            test_data = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/test_data.csv', delimiter=',')
            train_labels = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/train_labels.csv', delimiter=',')
            test_labels = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/test_labels.csv', delimiter=',')
        
        # if not, calculate mutual information
        else:
            "calculating mutual info"
            train_data = np.array([get_mutual_info(series, bins=mi_bin_num).flatten() for series in train_data])
            test_data = np.array([get_mutual_info(series, bins=mi_bin_num).flatten() for series in test_data])
    
    elif metric == "chow_liu":
        if os.path.exists(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}/data.csv'):
            print("Using precomputed chow-liu trees.")
            # load precomputed chow-liu trees
            train_data = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}/data.csv', delimiter=',')
            test_data = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}/test_data.csv', delimiter=',')
            train_labels = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}/train_labels.csv', delimiter=',')
            test_labels = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}/test_labels.csv', delimiter=',')
        
        elif os.path.exists(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/data.csv'):
            print("Using precomputed mutual information for chow-liu trees.")
            # load precomputed mutual information
            train_mi = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/data.csv', delimiter=',')
            test_mi = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/test_data.csv', delimiter=',')
            train_labels = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/train_labels.csv', delimiter=',')
            test_labels = np.loadtxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/mutual_info/{fraction}/test_labels.csv', delimiter=',')

            # reshape for msdl
            train_mi = train_mi.reshape(train_mi.shape[0], 39, 39)
            test_mi = test_mi.reshape(test_mi.shape[0], 39, 39)
            
            # get chow-liu trees
            train_data = np.array([get_chow_liu_tree(train_data[series_num], mi_matrix=train_mi[series_num], bins=mi_bin_num)[1].flatten() for series_num in range(len(train_data))])
            test_data = np.array([get_chow_liu_tree(test_data[series_num], mi_matrix=test_mi[series_num], bins=mi_bin_num)[1].flatten() for series_num in range(len(test_data))])
        
        else:
            "calculating mutual info chow-liu trees"
            train_data = np.array([get_chow_liu_tree(series, bins=mi_bin_num)[1].flatten() for series in train_data])
            test_data = np.array([get_chow_liu_tree(series, bins=mi_bin_num)[1].flatten() for series in test_data])
    else:
        raise ValueError("Invalid metric. Choose 'covariance', 'mutual_info', or 'chow-liu'.")

    # fit the model
    model.fit(train_data, train_labels)

    # make predictions
    train_predictions = model.predict(train_data)
    test_predictions = model.predict(test_data)

    # get accuracy
    data = {"train_data": train_data, "test_data": test_data, "train_labels": train_labels, "test_labels": test_labels}
    train_accuracy = accuracy_score(train_labels, train_predictions)
    test_accuracy = accuracy_score(test_labels, test_predictions)

    return data, train_accuracy, test_accuracy

def save_data(data, train_accuracy, test_accuracy, model_name = "gnb", metric="covariance", fraction=0.1, hold_out=0.2, bin_num=100, mi_bin_num=10):
    """
    Save the data and accuracies to CSV files.
    """
    # create directory if it doesn't exist
    if not os.path.exists(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}'):
        os.makedirs(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}')

    # save data
    np.savetxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}/data.csv', data['train_data'], delimiter=',')
    np.savetxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}/test_data.csv', data['test_data'], delimiter=',')
    
    # save labels
    np.savetxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}/train_labels.csv', data['train_labels'], delimiter=',')
    np.savetxt(f'{base_dir}/code/functional_connectivity/abide/output/data/{model_name}/{metric}/{fraction}/test_labels.csv', data['test_labels'], delimiter=',')
    
    # save accuracies
    with open(f'{base_dir}/code/functional_connectivity/abide/output/data//{model_name}/{metric}/{fraction}/accuracies.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['Metric', 'Model Name', 'Fraction', 'Hold Out', 'Bin Num', 'MI Bin Num', 'Train Accuracy', 'Test Accuracy'])
        writer.writerow([metric, model_name, fraction, hold_out, bin_num, mi_bin_num, train_accuracy, test_accuracy])
    print(f"Data and accuracies saved for metric: {metric}, fraction: {fraction}, hold_out: {hold_out}, bin_num: {bin_num}, mi_bin_num: {mi_bin_num}")

def run_model(tdc_timeseries, asd_timeseries, model_name="gnb", metric="covariance", fraction=0.1, hold_out=0.2, bin_num=1000, mi_bin_num=10):

    if model_name == "gnb":
        model = GaussianNB()
    elif model_name == "logistic":
        model = LogisticRegression(max_iter=1000)

    # Clearing the model
    model.theta_ = None
    model.sigma_ = None

    # get the data
    preproc_data = data_split(tdc_timeseries, asd_timeseries, fraction=fraction, hold_out=hold_out, bin_num=bin_num)

    # shuffle the labels
    labeled_data = shuffle_labels(preproc_data)

    # calculate metric and return processed data
    data, train_accuracy, test_accuracy = binary_prediction_model(labeled_data, model_name=model_name, model=model, metric=metric, fraction=fraction, mi_bin_num=mi_bin_num)

    save_data(data, train_accuracy, test_accuracy, model_name=model_name, metric=metric, fraction=fraction, hold_out=hold_out, bin_num=bin_num, mi_bin_num=mi_bin_num)

    return data, train_accuracy, test_accuracy


In [93]:
def run_experiment(fractions=[0.1], total_bin=1000):

    for fraction in fractions:
        # calculate bin_num based on fraction
        bin_num = fraction*total_bin
        print(f"Running experiment with fraction: {fraction}, bin_num: {bin_num}")
        # run the model with covariance
        run_model(tdc_timeseries, asd_timeseries, bin_num=bin_num, metric="covariance", fraction=fraction)
        # run the model with mutual information
        run_model(tdc_timeseries, asd_timeseries, metric="mutual_info", fraction=fraction, hold_out=0.2, bin_num=bin_num, mi_bin_num=10)
        # run the model with chow-liu
        run_model(tdc_timeseries, asd_timeseries, metric="chow_liu", fraction=fraction, hold_out=0.2, bin_num=bin_num, mi_bin_num=10)

In [98]:
run_experiment(fractions=[0.01, 0.02, 0.03, 0.04])

Running experiment with fraction: 0.01, bin_num: 10.0
Calculating covariance matrices.
Data and accuracies saved for metric: covariance, fraction: 0.01, hold_out: 0.2, bin_num: 10.0, mi_bin_num: 10
Data and accuracies saved for metric: mutual_info, fraction: 0.01, hold_out: 0.2, bin_num: 10.0, mi_bin_num: 10
Using precomputed mutual information for chow-liu trees.
Data and accuracies saved for metric: chow_liu, fraction: 0.01, hold_out: 0.2, bin_num: 10.0, mi_bin_num: 10
Running experiment with fraction: 0.02, bin_num: 20.0
Calculating covariance matrices.
Data and accuracies saved for metric: covariance, fraction: 0.02, hold_out: 0.2, bin_num: 20.0, mi_bin_num: 10
Data and accuracies saved for metric: mutual_info, fraction: 0.02, hold_out: 0.2, bin_num: 20.0, mi_bin_num: 10
Using precomputed mutual information for chow-liu trees.
Data and accuracies saved for metric: chow_liu, fraction: 0.02, hold_out: 0.2, bin_num: 20.0, mi_bin_num: 10
Running experiment with fraction: 0.03, bin_num: